In [1]:
using NBInclude
nbinclude("../src/PoissonProcessEstimation.ipynb")
import PoissonProcessEstimation

In [2]:
# Load the data
using MAT # to load mat file
# Load some data
file = matopen("../../MotorHam1.mat")
rawdata = read(file, "S")
size(rawdata) #neuron by timebin by trial
# read the data into our input format, trial x (neuron) x ((spiketimes)))
hasfired = zeros(size(rawdata,1),1) # track if a neuron has fired ever
data = Array(Any, (size(rawdata,3),))
for n = 1:length(data)
    data[n] = Array(Any, (size(rawdata,1),))
    for m = 1:size(rawdata,1)        
        data[n][m] = find(x->(x>=1), rawdata[m,201:1200,n])*1e-3 # set of spiketimes between 0 and 1 sec
        if length(data[n][m]) > 0
            hasfired[m] = 1
        end
    end
end

# trim the data of neurons that are silent on every trial (no spikes whatsoever within the whole dataset)
for n = 1:length(data)
    data[n] = data[n][find(x->x==1, hasfired)]
end
        

In [15]:
# Set up kernels
KernList = [MLKernels.GaussianKernel(400.0), MLKernels.GaussianKernel(200.0), MLKernels.GaussianKernel(200.0), MLKernels.GaussianKernel(100.0)]

4-element Array{MLKernels.ExponentialKernel{Float64,:γ1},1}:
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=400.0,γ=1.0)
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=200.0,γ=1.0)
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=200.0,γ=1.0)
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=100.0,γ=1.0)

In [16]:
size(data)

(216,)

In [17]:
# Initialise params
θ = PoissonProcessEstimation.create_rand_params(216,198,length(KernList),[30,20,20,10]);
# Symmetry breaking of C
θ.C = θ.C + randn(size(θ.C))*0.05;

In [18]:
θ_orig = θ
θ_opt = deepcopy(θ)
C, α, u, N, M, D, J = PoissonProcessEstimation.name_params(θ_opt);

In [19]:
#PoissonProcessEstimation.Cost(data, θ_opt, KernList)

In [20]:
@time Ktu, dtKtu, ddtKtu, Kuu, I = 
    PoissonProcessEstimation.optimise!(data, θ_opt, KernList, η_RKHS=1e1, η_Cm=1e1, num_iters = 5);

 91.126472 seconds (2.38 G allocations: 37.503 GB, 4.22% gc time)
Kernels have been built

Starting optimisation, initial cost is [47973.19118365363]

Starting iter 1
  4.295274 seconds (31.35 M allocations: 1.637 GB, 5.65% gc time)
Alphas inferred, cost is [9931.286745337164]
  5.308938 seconds (55.57 M allocations: 1.870 GB, 4.20% gc time)
C learned, cost is [8883.472811560738]

Starting iter 2
  2.250016 seconds (29.74 M allocations: 1.565 GB, 9.93% gc time)
Alphas inferred, cost is [11030.666410906197]
  5.251409 seconds (55.43 M allocations: 1.863 GB, 4.25% gc time)
C learned, cost is [11520.994766681553]

Starting iter 3
  2.221563 seconds (29.74 M allocations: 1.565 GB, 9.84% gc time)
Alphas inferred, cost is [13880.942887457732]
  5.341918 seconds (55.43 M allocations: 1.863 GB, 4.21% gc time)
C learned, cost is [14215.466421026416]

Starting iter 4
  2.080391 seconds (29.74 M allocations: 1.565 GB, 10.27% gc time)
Alphas inferred, cost is [14971.968915499163]
  5.288392 second

In [21]:
using JLD
save("MotorHam1_results_kernel_all_closedform.jld", "theta_opt", θ_opt, "data", data, "theta", θ, "KernList", KernList)

In [22]:
PoissonProcessEstimation.plot_params(data, θ_opt, θ_orig, KernList, num_neur=15, trials=1:50, dim_latent=4)

In [36]:
[θ_opt.C[[32,197],:]]

 in depwarn at deprecated.jl:73
 in oldstyle_vcat_warning at /opt/homebrew-cask/Caskroom/julia/0.4.5/Julia-0.4.5.app/Contents/Resources/julia/lib/julia/sys.dylib
 in vect at abstractarray.jl:32
 in include_string at loading.jl:282
 in execute_request at /Users/gergobohner/.julia/v0.4/IJulia/src/execute_request.jl:164
 in eventloop at /Users/gergobohner/.julia/v0.4/IJulia/src/IJulia.jl:138
 in anonymous at task.jl:447
while loading In[36], in expression starting on line 1


2x4 Array{Float64,2}:
 -0.0582204   0.551006  0.157829  -0.806259
 -0.61153    -0.7151    0.752535   0.889687

In [25]:
eig(θ_opt.C'*θ_opt.C)

([42.91361383590916,118.05329975998075,163.69446623181858,608.6920419336427],
4x4 Array{Float64,2}:
  0.00959584  -0.021133   0.230543  -0.972785 
  0.026342    -0.676143  -0.719669  -0.155608 
 -0.533509     0.612998  -0.562605  -0.151913 
  0.84533      0.408188  -0.335265  -0.0799843)

In [ ]:
ddtKtu[3]

In [ ]:
I

In [ ]:
PoissonProcessEstimation.opti